<a href="https://colab.research.google.com/github/aarakova/VvAD/blob/SAD4/%D0%A1%D0%9E%D0%98_%D0%9B%D0%A04_%D0%A0%D0%B0%D0%BA%D0%BE%D0%B2%D0%B0_4218.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from scipy.stats import skew, kurtosis, shapiro, chi2_contingency, epps_singleton_2samp, normaltest
import numpy as np
from math import sqrt, exp

In [ ]:
n, m = 7, 3

data = {
    "№ предприятия": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    "Выпуск продукции": [60 + n, 78, 41, 54, 60 + n, n * m + 20, 45, 57, 67, 80 + n, 92, 48, 59, 68, 80 + n, 52, 62, 69, 85, 70 + n, 71, n * m + 30, 72, 88, 70 + n, 74, 96, 75, 101, 70 + n],
    "Прибыль": [15.7, 18.0, 12.1, 13.8, 15.5, n + m + 10, 12.8, 14.2, 15.9, 17.6, 18.2, n + m + 5, 16.5, 16.2, 16.7, 14.6, 14.8, 16.1, 16.7, 15.8, 16.4, n + m + 20, 16.5, 18.5, 16.4, 16.0, 19.1, 16.3, 19.6, 17.2]
}

# 1. Коэффициент асимметрии и эксцесса для 'Выпуск продукции' и 'Прибыль'


In [ ]:
skewness_vp = skew(data["Выпуск продукции"])
kurtosis_vp = kurtosis(data["Выпуск продукции"], fisher=True)  # Fisher=True означает избыточный эксцесс

skewness_profit = skew(data["Прибыль"])
kurtosis_profit = kurtosis(data["Прибыль"], fisher=True)

In [ ]:
summary = {
    "Асимметрия и Эксцесс": {
        "Выпуск продукции": {"Асимметрия": skewness_vp, "Эксцесс": kurtosis_vp},
        "Прибыль": {"Асимметрия": skewness_profit, "Эксцесс": kurtosis_profit}
    }}
summary

{'Асимметрия и Эксцесс': {'Выпуск продукции': {'Асимметрия': -0.04742232594659079,
   'Эксцесс': -0.7682051731287745},
  'Прибыль': {'Асимметрия': 2.586958120664307, 'Эксцесс': 9.472712901290855}}}

#### Коэффициент асимметрии и эксцесса:

- **Для 'Выпуск продукции':**
  - Асимметрия: -0.047 (приближается к 0, что указывает на близость к симметричному распределению)
  - Эксцесс: -0.768 (показывает, что распределение имеет меньше экстремальных значений по сравнению с нормальным)

- **Для 'Прибыли':**
  - Асимметрия: 2.587 (значительное смещение влево, указывает на сильную асимметрию)
  - Эксцесс: 9.473 (распределение имеет больше пиковых значений по сравнению с нормальным)

# 2. Критерий хи-квадрат для 'Выпуск продукции' и 'Прибыль' (разбиение данных на интервалы равной ширины)


In [ ]:
observed_vp, bins_vp = np.histogram(data["Выпуск продукции"], bins='auto')
expected_vp = np.full_like(observed_vp, np.mean(observed_vp))
chi2_vp, p_chi2_vp = chi2_contingency([observed_vp, expected_vp])[:2]

In [ ]:
observed_profit, bins_profit = np.histogram(data["Прибыль"], bins='auto')
expected_profit = np.full_like(observed_profit, np.mean(observed_profit))
chi2_profit, p_chi2_profit = chi2_contingency([observed_profit, expected_profit])[:2]

In [ ]:
summary = {
    "Критерий хи-квадрат": {
        "Выпуск продукции": {"Статистика хи-квадрат": chi2_vp, "p-значение": p_chi2_vp},
        "Прибыль": {"Статистика хи-квадрат": chi2_profit, "p-значение": p_chi2_profit}
    }}
summary

{'Критерий хи-квадрат': {'Выпуск продукции': {'Статистика хи-квадрат': 1.5054390054390054,
   'p-значение': 0.9124413025475099},
  'Прибыль': {'Статистика хи-квадрат': 23.466666666666665,
   'p-значение': 0.05308738743166657}}}

In [ ]:
print("Наблюдаемые частоты для 'Выпуск продукции':", observed_vp)
print("Ожидаемые частоты для 'Выпуск продукции':", expected_vp)
print("Наблюдаемые частоты для 'Прибыль':", observed_profit)
print("Ожидаемые частоты для 'Прибыль':", expected_profit)

Наблюдаемые частоты для 'Выпуск продукции': [4 5 6 8 4 3]
Ожидаемые частоты для 'Выпуск продукции': [5 5 5 5 5 5]
Наблюдаемые частоты для 'Прибыль': [ 2  2  4 13  3  3  2  0  0  0  0  0  0  0  1]
Ожидаемые частоты для 'Прибыль': [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


Критерий хи-квадрат: не работает из-за того, что есть частоты которые меньше 5, следовательно тест статистически некоректен.

# 3. Тест Шапиро-Уилка


In [ ]:
shapiro_vp_stat, shapiro_vp_p = shapiro(data["Выпуск продукции"])
shapiro_profit_stat, shapiro_profit_p = shapiro(data["Прибыль"])

In [ ]:
summary = {
    "Тест Шапиро-Уилка": {
        "Выпуск продукции": {"Статистика": shapiro_vp_stat, "p-значение": shapiro_vp_p},
        "Прибыль": {"Статистика": shapiro_profit_stat, "p-значение": shapiro_profit_p}
    }
}
summary

{'Тест Шапиро-Уилка': {'Выпуск продукции': {'Статистика': 0.9758151962861061,
   'p-значение': 0.7067420543890376},
  'Прибыль': {'Статистика': 0.7529934372752245,
   'p-значение': 1.0183221562102387e-05}}}

#### Критерий Шапиро-Уилка:

- **Для 'Выпуск продукции':** p-value = 0.707 (нет оснований отвергнуть гипотезу о нормальности)
- **Для 'Прибыли':** p-value = 0.00001 (существенное отклонение от нормальности)


# 4. Тест Эппса-Палли


In [ ]:
def epps_palli_statistic(X):
    N = len(X)
    X = np.array(X)
    X_mean = X.mean()

    m2 = np.var(X, ddof=0)

    A = sqrt(2) * np.sum([exp(-(X[i] - X_mean) ** 2 / (4 * m2)) for i in range(N)])

    B = 2 / N * np.sum(
        [np.sum([exp(-(X[j] - X[k]) ** 2 / (2 * m2)) for j in range(0, k)])
         for k in range(1, N)]
    )

    TEP_calc = 1 + N / sqrt(3) + B - A
    return TEP_calc

tep_vp = epps_palli_statistic(data["Выпуск продукции"])
tep_profit = epps_palli_statistic(data["Прибыль"])

epps_palli_results = {
    "Критерий Эппса-Палли": {
        "Выпуск продукции": {"Статистика": tep_vp},
        "Прибыль": {"Статистика": tep_profit}
    }
}

epps_palli_results

{'Критерий Эппса-Палли': {'Выпуск продукции': {'Статистика': 0.04897058188379333},
  'Прибыль': {'Статистика': 1.1840135929373474}}}

### Вывод по критерию Эппса-Палли

- **Для 'Выпуск продукции':**  
  Статистика критерия Эппса-Палли составила 0.04897, что является низким значением и указывает на близкое соответствие распределения данных нормальному. Это говорит о том, что отклонение от нормальности несущественно.

- **Для 'Прибыли':**  
  Статистика критерия Эппса-Палли составила 1.184, что значительно выше по сравнению с 'Выпуском продукции'. Это значение указывает на заметное отклонение от нормального распределения.


# Вывод:

### Для 'Выпуск продукции':
- **Критерий хи-квадрат**: Тест статистически некорректен, поскольку некоторые ожидаемые частоты меньше 5. Это нарушает одно из ключевых условий применения данного теста.
- **Гипотеза нормальности**: Большинство других тестов подтверждают гипотезу о нормальности данных. Коэффициенты асимметрии и эксцесса указывают на незначительное отклонение. Тест Шапиро-Уилка также поддерживает гипотезу нормальности.
- **Общий вывод**: Данные по 'Выпуск продукции' можно считать соответствующими нормальному распределению, несмотря на невозможность применения критерия хи-квадрат.

### Для 'Прибыли':
- **Критерий хи-квадрат**: Не работает из-за наличия частот меньше 5, что делает тест статистически некорректным.
- **Гипотеза нормальности**: Данные демонстрируют значительное отклонение от нормального распределения. Это подтверждается высоким значением асимметрии и эксцесса, а также низким p-значением теста Шапиро-Уилка.
- **Общий вывод**: Данные по 'Прибыль' не соответствуют нормальному распределению
